<img src="./img/img-2.png"><br> 


## データセット
本編で扱う 『茶の元素分析データセット』（data.xlsx）、は、ブラックセイロン(BC)、ブラックトルコ(BT)、グリーンセイロン(GC)、グリーントルコ(GT)の4種類の茶葉について、3つの濃度（1%、2%、3%）で抽出したサンプルの誘導結合プラズマ発光分光分析法（ICP-OES）の元素分析データです。[1]

本コードは、Optunaを使用して多目的最適化のベイズ最適化を行います。目的関数でFeの最大化、Alの最小化、Mnの安定化を定義し、最適化パラメータとしてConcentration、time、およびOneHotエンコーディングされたteaを使用します。目的関数は、これらのパラメータに基づき、学習済みの線形回帰モデル（MLR）および人工ニューラルネットワーク（ANN）で予測を行い、複数の目的を同時に最適化します。

[1] Durmus, Y., Atasoy, A.D. & Atasoy, A.F. Mathematical optimization of multilinear and artificial neural network regressions for mineral composition of different tea types infusions. Sci Rep 14, 18285 (2024). https://doi.org/10.1038/s41598-024-69149-1

---
<br>  
<img src="./img/main_image.jpg" width="50%">
<br> 

---

**Chemical composition** 
* 9種類の元素分析値（Al, Ca, Cu, Fe,K,Mg,Mn,Na,Zn） 

**Experimental parameters**  
* Concentration：　お茶の濃度
* time:　抽出時間

**Label**
* grouped:　品種・色・濃度・抽出時間を識別できる一種の飼料名（MLは使用せず） 
* teaConc:　濃度ラベル（MLは使用せず）
* tea_org:　色ラベル（MLは使用せず）
* tea_var:　品種ラベル（MLは使用せず）
* tea：　　 色と品種ラベル（MLで使用）

### 教材への接続
google colabにおけるオンラインの場合にこのラインを実行します。（<font color="red">Google colabに接続しない場合には不要</font>）

In [ ]:
!pip install optuna
!!git clone https://github.com/ARIM-Usecase/Example_2.git
%cd Example_2

### ライブラリのインポート
コード実行で必要なpythonのライブラリをimport文でロードします。

In [22]:
#汎用ライブラリ
import pandas as pd

# 機械学習
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression

#深層学習
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LeakyReLU
from tensorflow.keras.optimizers import Adam

# 回帰分析
from sklearn.linear_model import LinearRegression
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# TPE最適化（Optuna）
import optuna
from optuna.samplers import TPESampler

## 1.データセットの読み込みと前処理


### サンプルファイルの読み込み
[data]フォルダーにあるdata.xlsxファイルをDataFrameとして読み込み、その結果をdfという変数に格納します。

In [2]:
# データの読み込み
df= pd.read_excel('./data/data.xlsx', header = 1)
df

,Al,Ca,Cu,Fe,K,Mg,Mn,Na,Zn,grouped,tea,Concentration,time,teaConc,tea_org,tea_var
0,3.297,4.356,0.031290,0.067,99.06,3.531,1.455,0.541,0.131,Black Turkish 1 2,BT,1,2,BT1,black,turki
1,4.267,4.118,0.031290,0.079,106.50,3.378,1.542,0.603,0.126,Black Turkish 1 2,BT,1,2,BT1,black,turki
2,4.088,4.763,0.033370,0.084,114.00,4.763,1.838,1.058,0.156,Black Turkish 1 5,BT,1,5,BT1,black,turki
3,4.338,4.556,0.033370,0.091,122.60,5.005,2.269,0.958,0.162,Black Turkish 1 5,BT,1,5,BT1,black,turki
4,4.732,5.138,0.035514,0.110,132.40,5.626,2.998,1.510,0.165,Black Turkish 1 10,BT,1,10,BT1,black,turki
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,16.690,8.895,0.153000,0.236,323.40,20.450,10.420,6.360,0.335,Green Ceylan 3 30,GC,3,30,GC3,green,ceylon
164,17.620,8.909,0.177000,0.261,334.20,23.486,11.330,7.133,0.351,Green Ceylan 3 45,GC,3,45,GC3,green,ceylon
165,17.920,9.056,0.180000,0.266,332.30,22.840,11.290,7.609,0.358,Green Ceylan 3 45,GC,3,45,GC3,green,ceylon
166,17.820,9.128,0.175000,0.273,367.30,24.560,12.110,8.537,0.372,Green Ceylan 3 60,GC,3,60,GC3,green,ceylon


## 2.　前処理
### カテゴリカル変数のOne Hot Encoding化
Scikit-learnのOneHotEncoderクラスを使用して、質的変数のお茶の銘柄「tea」列の値をワンホットエンコーディング化します。

In [3]:
# OneHotエンコーディング
encoder = OneHotEncoder(drop='first', sparse_output=False)
df_encoded = encoder.fit_transform(df[['tea']])

# エンコードされたteaをデータフレームに追加
encoded_columns = encoder.get_feature_names_out(['tea'])
df[encoded_columns] = df_encoded

### 説明変数と目的変数
データフレームdfから、茶の濃度（Concentration）と浸出時間（time）に加えて、One Hot Encodingされたお茶の銘柄を説明変数とします。一方、目的変数はベイズ最適化として鉄（Fe）、アルミニウム（Al）、およびマンガン（Mn）の元素分析値を定義します。

In [4]:
# 特徴量とターゲットの分離
X = df[['Concentration', 'time'] + list(encoded_columns)]
y_Fe = df['Fe']
y_Al = df['Al']
y_Mn = df['Mn']

### 標準化  
Scikit-learnの`StandardScaler`クラスで、データの標準化を行います。

In [5]:
# データの標準化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## 3.ベイズ最適化
### 3.1 MLRモデルによる最適化
LinearRegression()クラスで、鉄（Fe）、アルミニウム（Al）、およびマンガン（Mn）の元素分析値の線形重回帰モデルを構築します。

In [6]:
model_Fe = LinearRegression()
model_Al = LinearRegression()
model_Mn = LinearRegression()

model_Fe.fit(X_scaled, y_Fe)
model_Al.fit(X_scaled, y_Al)
model_Mn.fit(X_scaled, y_Mn)

LinearRegression()

### Optuna目的関数の定義
Optunaを用いて多目的最適化を行います。Concentration、time、およびカテゴリ変数tea（OneHotエンコーディング適用）を最適化パラメータとして取り扱います。スケーリング後、事前学習済みの線形回帰モデルでFeを最大化、Alを最小化、Mnを安定化させ、加えてそれらの差分も評価指標に加えた複合的な目標値を出力します。

In [7]:
# 目的関数の定義 (MLR 用)
def objective_mlr(trial):
    # 試行による最適化パラメータ
    X_trial = pd.DataFrame({
        'Concentration': [trial.suggest_float('Concentration', 1.0, 3.0)],
        'time': [trial.suggest_float('time', 2.0, 60.0)],
        'tea': [trial.suggest_categorical('tea', ['BT','BC', 'GC', 'GT'])]
    })

    # OneHotエンコーディングの適用
    X_encoded_trial = encoder.transform(X_trial[['tea']])
    encoded_columns = encoder.get_feature_names_out(['tea'])
    X_trial[encoded_columns] = X_encoded_trial

    # エンコーディング後の列のみを使用
    X_trial = X_trial[['Concentration', 'time'] + list(encoded_columns)]

    # 試行データのスケーリング（学習時のスケーラーを適用）
    X_scaled_trial = scaler.transform(X_trial)

    # 予測
    y_pred_Fe = model_Fe.predict(X_scaled_trial)[0]
    y_pred_Al = model_Al.predict(X_scaled_trial)[0]
    y_pred_Mn = abs(model_Mn.predict(X_scaled_trial)[0] - 5.3)

    # 目的関数の計算（Fe の最大化、Al の最小化、Mn の安定化）
    return y_pred_Fe, y_pred_Al, y_pred_Mn, y_pred_Fe - y_pred_Al - y_pred_Mn
  

### Optuna最適化実行
Optunaによる最適化プロセスでは、試行ごとに提案されるパラメータを基に、多目的関数objective_mlrを評価します。最適化の方向は、Feの最大化、Alの最小化、Mnの安定化（特定値からの偏差の最小化）、およびこれらの差分の最小化を含む4つの目的で構成されています。Tree-structured Parzen estimator (TPE)のサンプラーであるTPESamplerを使用して、300回の試行で効率的に探索します。

In [8]:
# Optunaによる最適化 (MLR)
study_mlr = optuna.create_study(
    directions=["maximize", "minimize","minimize","minimize"],
    sampler=TPESampler()
)

study_mlr.optimize(objective_mlr, n_trials=300)

[I 2025-01-04 11:54:55,935] A new study created in memory with name: no-name-2b458e80-6203-4293-964a-b7d4c476e39a
[I 2025-01-04 11:54:55,941] Trial 0 finished with values: [0.16535168674724732, 8.802612252915756, 1.2085838764069665, -9.845844442575476] and parameters: {'Concentration': 1.6374383341541932, 'time': 33.654834682681006, 'tea': 'BT'}.
[I 2025-01-04 11:54:55,947] Trial 1 finished with values: [0.1982605181721695, 10.131987235924894, 2.9299314399132355, -12.86365815766596] and parameters: {'Concentration': 1.5941767144079568, 'time': 56.25640881007688, 'tea': 'BT'}.
[I 2025-01-04 11:54:55,952] Trial 2 finished with values: [0.15428920730048226, 13.449296949961784, 1.9694044741010872, -15.264412216762388] and parameters: {'Concentration': 2.707718816918577, 'time': 8.090154041306981, 'tea': 'GT'}.
[I 2025-01-04 11:54:55,959] Trial 3 finished with values: [0.17240559867235183, 10.090719143007998, 2.0027584200184476, -11.921071964354095] and parameters: {'Concentration': 1.29606

### 可視化
Optunaの最適化履歴をstudy_mlrから得られる試行データをもとに、第一目的（Feの最大化）に対応する履歴をプロットします。

In [9]:
# 最適化履歴をプロット
fig = optuna.visualization.plot_optimization_history(
    study_mlr,
    target=lambda t: t.values[0],  # Feを選択
    target_name="Objective Fe_Conc"     # ラベル名を設定
)

fig.update_layout(width=800, height=600, showlegend=False)  # 凡例を非表示に設定
fig.show()

### カウンタープロット
Optunaの最適化履歴をカウンタープロット（等高線）で可視化するものです。Concentrationとtimeの2つのパラメータに対して最適化の履歴を描画し、第一の目的（Feの最大化）の値に基づいて等高線を作成します。

In [10]:
# 最適化履歴をプロット（Concentration と time のみ）
fig = optuna.visualization.plot_contour(
    study_mlr,
    params=["Concentration", "time"],  # プロット対象のパラメータを指定
    target=lambda t: t.values[0],      # 第一の目的を選択
    target_name="Objective 1"         # ラベル名を設定
)

# レイアウト調整
fig.update_layout(width=800, height=600, showlegend=False)  # 凡例を非表示に設定
fig.show()

### スライスプロット
Optunaの最適化結果をスライスプロットで可視化するものです。第一の目的（Feの最大化）の結果に基づいて、最適化パラメータに対するスライス（1つのパラメータの変化に対する目的関数の変化）を描画します。

In [11]:
# 結果のプロット (スライスプロット)
fig = optuna.visualization.plot_slice(
    study_mlr,
    target=lambda t: t.values[0],      # 第一の目的を選択
    target_name="Objective 1"         # ラベル名を設定
)

# レイアウト調整
fig.update_layout(width=1200, height=600, showlegend=False)  # 凡例を非表示に設定
fig.show()

### 3.2 ANNモデルによる最適化
Kerasを用いて鉄（Fe）、アルミニウム（Al）、およびマンガン（Mn）の元素分析値の人工ニューラルネットワーク（ANN）モデルを構築します。ANNモデルは事前にbuild_ann_moedel()関数を定義し、都度、呼び出すようにします。

In [25]:
# 人工ニューラルネットワークの関数化
def build_ann_model(input_dim):
    
    model = Sequential()
    model.add(Input(shape=(input_dim,))) 
    model.add(LeakyReLU())
    model.add(Dense(18))
    model.add(LeakyReLU())
    model.add(Dense(8))
    model.add(LeakyReLU())
    model.add(Dense(12))
    model.add(LeakyReLU())
    model.add(Dense(18))
    model.add(LeakyReLU())
    model.add(Dense(1))
    
    model.compile(optimizer=Adam(learning_rate=0.0112), loss='mse')
    
    return model

In [26]:
ann_model_Fe = build_ann_model(X_scaled.shape[1])
ann_model_Al = build_ann_model(X_scaled.shape[1])
ann_model_Mn = build_ann_model(X_scaled.shape[1])

ann_model_Fe.fit(X_scaled, y_Fe, epochs=100, verbose=0)
ann_model_Al.fit(X_scaled, y_Al, epochs=100, verbose=0)
ann_model_Mn.fit(X_scaled, y_Mn, epochs=100, verbose=0)

### Optuna目的関数の定義
Optunaを用いて多目的最適化を行います。Concentration、time、およびカテゴリ変数tea（OneHotエンコーディング適用）を最適化パラメータとして取り扱います。ここでは定義した機械学習モデル以外はMLRと同じ記法です。

In [27]:
# 目的関数の定義 (ANN 用)
def objective_ann(trial):
    # 試行による最適化パラメータ
    X_trial = pd.DataFrame({
        'Concentration': [trial.suggest_float('Concentration', 1.0, 3.0)],
        'time': [trial.suggest_float('time', 2.0, 60.0)],
        'tea': [trial.suggest_categorical('tea', ['BT','BC','GC','GT'])]
    })

    # OneHotエンコーディングの適用
    X_encoded_trial = encoder.transform(X_trial[['tea']])
    encoded_columns = encoder.get_feature_names_out(['tea'])
    X_trial[encoded_columns] = X_encoded_trial

    # エンコーディング後の列のみを使用
    X_trial = X_trial[['Concentration', 'time'] + list(encoded_columns)]

    # 試行データのスケーリング（学習時のスケーラーを適用）
    X_scaled_trial = scaler.transform(X_trial)

    # 予測
    y_pred_Fe = ann_model_Fe.predict(X_scaled_trial)[0]
    y_pred_Al = ann_model_Al.predict(X_scaled_trial)[0]
    y_pred_Mn = abs(ann_model_Mn.predict(X_scaled_trial)[0] - 5.3)

    # 目的関数の計算（Fe の最大化、Al の最小化、Mn の安定化）
    return y_pred_Fe, y_pred_Al, y_pred_Mn, y_pred_Fe - y_pred_Al - y_pred_Mn

### Optuna最適化実行
最適化プロセスでも試行回数を100回とした以外はMLRと設定は同じです。

In [28]:
# Optunaによる最適化 (MLR)
study_ann = optuna.create_study(
    directions=["maximize", "minimize","minimize","minimize"],
    sampler=TPESampler()
)

study_ann.optimize(objective_ann, n_trials=100)

[I 2025-01-04 12:17:12,211] A new study created in memory with name: no-name-bbbc3e6f-d7ca-44df-937c-ad309d7d4d1d


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


[I 2025-01-04 12:17:12,477] Trial 0 finished with values: [0.05886942148208618, 2.5099620819091797, 4.096280097961426, -6.547372817993164] and parameters: {'Concentration': 1.016453332634255, 'time': 7.320709472229836, 'tea': 'BC'}.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step


[I 2025-01-04 12:17:12,646] Trial 1 finished with values: [0.07405874133110046, 3.2176153659820557, 2.4483869075775146, -5.591943740844727] and parameters: {'Concentration': 1.2086195587037796, 'time': 36.27233728790079, 'tea': 'BC'}.


...

[I 2025-01-04 12:18:05,356] Trial 97 finished with values: [0.1913682371377945, 15.540860176086426, 3.462893486022949, -18.81238555908203] and parameters: {'Concentration': 2.8731367723976873, 'time': 11.998873660013203, 'tea': 'GT'}.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


[I 2025-01-04 12:18:06,040] Trial 98 finished with values: [0.21702788770198822, 19.211315155029297, 6.755151748657227, -25.749439239501953] and parameters: {'Concentration': 2.7664337347882717, 'time': 35.35955212151484, 'tea': 'GT'}.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step


[I 2025-01-04 12:18:06,650] Trial 99 finished with values: [0.26187723875045776, 17.099550247192383, 5.464374542236328, -22.302047729492188] and parameters: {'Concentration': 2.511918548528273, 'time': 57.79763726626125, 'tea': 'GC'}.


### 可視化
Optunaの最適化履歴をstudy_mlrから得られる試行データをもとに、第一目的（Feの最大化）に対応する履歴をプロットします。

In [29]:
# 最適化履歴をプロット
fig = optuna.visualization.plot_optimization_history(
    study_ann,
    target=lambda t: t.values[0],  # Feを選択
    target_name="Objective Fe_Conc"     # ラベル名を設定
)

fig.update_layout(width=800, height=600, showlegend=False)  # 凡例を非表示に設定
fig.show()


### カウンタープロット
Optunaの最適化履歴をカウンタープロット（等高線）で可視化するものです。Concentrationとtimeの2つのパラメータに対して最適化の履歴を描画し、第一の目的（Feの最大化）の値に基づいて等高線を作成します。

In [30]:
# 最適化履歴をプロット（Concentration と time のみ）
fig = optuna.visualization.plot_contour(
    study_ann,
    params=["Concentration", "time"],  # プロット対象のパラメータを指定
    target=lambda t: t.values[0],      # 第一の目的を選択
    target_name="Objective 1"         # ラベル名を設定
)

# レイアウト調整
fig.update_layout(width=800, height=600, showlegend=False)  # 凡例を非表示に設定
fig.show()


### スライスプロット
Optunaの最適化結果をスライスプロットで可視化するものです。第一の目的（Feの最大化）の結果に基づいて、最適化パラメータに対するスライス（1つのパラメータの変化に対する目的関数の変化）を描画します。

In [31]:
# 結果のプロット (スライスプロット)
fig = optuna.visualization.plot_slice(
    study_ann,
    target=lambda t: t.values[0],      # 第一の目的を選択
    target_name="Objective 1"         # ラベル名を設定
)

# レイアウト調整
fig.update_layout(width=1200, height=600, showlegend=False)  # 凡例を非表示に設定
fig.show()